In [1]:
from subprocess import Popen, PIPE, STDOUT


In [2]:
name = input()

In [3]:
name

'hola'

In [5]:
a = []
for _ in range(1,3):
    a.append(input())
    

In [6]:
a

['1', '2']

In [28]:
from subprocess import Popen, PIPE, STDOUT
p = Popen(['/bin/sh'], stdout=PIPE, stdin=PIPE, stderr=PIPE,         shell=True,)
# stdout_data = p.communicate(input='ls'.encode())[0]

In [29]:
p.stdin.write('ls -la'.encode())

6

In [30]:
p.stdout.flush()


In [31]:
p.stdout.read(6)


KeyboardInterrupt: 

In [14]:
stdout_data = p.communicate(input='ls -la'.encode())[0]

ValueError: Cannot send input after starting communication

In [75]:
from threading import Thread
from threading import Timer

def run_with_timeout(timeout, default, f, *args, **kwargs):
    if not timeout:
        return f(*args, **kwargs)
    try:
        timeout_timer = Timer(timeout,  )
        timeout_timer.start()
        result = f(*args, **kwargs)
        return result
    except KeyboardInterrupt:
        return default
    finally:
        timeout_timer.cancel()

In [83]:
import subprocess


def start(executable_file):
    
    return subprocess.Popen(
        executable_file,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        close_fds=True,

)


def read(process):
    while True:
        try:
            line = outq.get(block=False)
            print('got line from outq: {0}'.format(line), end='')
        except queue.Empty:
            # print('could not get line from queue')
            break
    # return process.stdout.readline().strip()


def write(process, message):
    process.stdin.write(f"{message}\n".encode())
    process.stdin.flush()


def terminate(process):
    process.stdin.close()
    process.terminate()
    process.wait(timeout=0.2)


process = start("/bin/sh")
write(process, "ls -la")
# print(read(process))
write(process, "ls -la")
# terminate(process)

In [13]:
from queue import Queue, Empty
import threading
import time
import subprocess

class Debugger:
    std_q = Queue()
    err_q = Queue()
    process = None
    lock = False
    
    def __init__(self, problem) -> None:
        
        self.process = subprocess.Popen(
        ["../up_siadex/bin/planner","-d","../up_siadex//test/domain.hpdl","-p","..//up_siadex//test/problem.hpdl","-g"],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        # close_fds=True,
        )
        def output_reader(proc, outq):
            for line in iter(proc.stdout.readline, b''):
                outq.put(line.decode('utf-8'))
        
        def error_reader(proc, outq):
            for line in iter(proc.stderr.readline, b''):
                outq.put(line.decode('utf-8'))
                
        t = threading.Thread(target=output_reader, args=(self.process, self.std_q))
        t2 = threading.Thread(target=error_reader, args=(self.process, self.err_q))
        t.start()
        t2.start()

    def start(self):
        pass

        
    def command(self,c):
        # If the process is locked running previous commands, lets wait until it has finnished
        while self.lock:
            time.sleep(0.5)
        try:
            # Lock the process
            self.lock = True
    
            # Write the command in the input
            self.process.stdin.write(f"{c}\n".encode())
            self.process.stdin.flush()
            # Time to wait for the response        
            time.sleep(1)
            
            std = self._capture_std()
            err = self._capture_error()
            [print(msg, end='') for msg in std]
            print("_"*50)
            [print(msg, end='') for msg in err]
        except BrokenPipeError as e:
            print("Restarting")
            self.__init__()
            self.command(c)
        
        
    def _capture_output(self, q: Queue):
        """This methods capture the output from a thread."""
        self.lock = True
        result = []
        while True:
            try:
                # Capture msgs
                line = q.get(block=False)
                result.append(line)
                # print(line, end='')
            except Empty:
                # No more messages so... 
                # # Free the lock 
                # # return the result
                self.lock = False
                return result
        
        
        
    def _capture_std(self):
        """This methods captures the output of STD from the thread."""
        return self._capture_output(self.std_q)
            
    def _capture_error(self):
        """This methods captures the output of err from the thread."""
        return self._capture_output(self.err_q)
        

In [18]:
debugger = Debugger()
debugger.command("help describe")

debug:> help describe
__________________________________________________

(*** 1 ***) Initializating planning process...
(*** 1 ***) Initializating python...
(*** 1 ***) Depth: 1
(*** 1 ***) Selecting task to expand from agenda.

(*** 1 ***) Initializating planning process...
(*** 1 ***) Initializating python...
(*** 1 ***) Depth: 1
(*** 1 ***) Selecting task to expand from agenda.
Command: `describe'.

Description: Shows detailed information about a structure defined in the domain.
	`describe <predicate-expression>':	Print the description in the domain relative to the predicate expression. It can be a predicate or a task.
See also: `print', `display'


In [19]:
debugger.command("""help eval""")

debug:> help eval
__________________________________________________
Command: `eval <precondition>'.

Description: Evaluates the given expression and prints the produced unifications.


In [20]:
debugger.command("help print")


debug:> help print
__________________________________________________
Command: `print'. Shortcut: `p'

Description: Prints information about the planning context.
	`print state':	Prints the current state.
	`print agenda':	Prints the current agenda.
	`print plan':	Prints the ongoing plan.
	`print options':	Prints the options you can peform on next step.
	`print termtable':	Prints the internal term table.
	`print tasks':	Prints all the tasks defined in the domain.
	`print predicates':	Prints all the avaliable predicates defined in the domain.
	`print <predicate-expression>':	Prints all the predicates in the current state, or the tasks in current plan, that match the given expression.


In [21]:
debugger.command("help")


debug:> help
__________________________________________________
Type `help <command>' to obtain detailed information about a command.
Avaliable commands:
print          display        quit           plot
continue       undisplay      next           set
describe       break          disable        enable
nexp           eval           watch          apply
<enter> executes the last command.


In [75]:
debugger.command("help break")

debug:> help break
Command: `break'. Shortcut `b'

Description: Manages the stablished breakpoints.
	`break':	Lists all defined breakpoints.
	`break <number>':	Prints breakpoint whith given id.
	`break <predicate>':	Defines a new breakpoint. <predcate> can be a task definition or a simple predicate.
See also: `watch', `disable', `enable'.


In [74]:
debugger.command("print agenda")


debug:> print agenda
Succesors:
[0] 9 
[1] 0 
[2] 3 
[3] 0 
[4] 5 
[5] 6 
[6] 7 
[7] 3 
[8] 5 
[9] 5 
Task list: 
[1] (closed)     :unexpanded (solve_elevator)
[2] (closed)     :unexpanded (deliver_person p0 f0 f1)
[3] (pending)    :unexpanded (solve_elevator)
[4] (closed)     :unexpanded (move f0 f0)
[5] (agenda)     :unexpanded (board p0 f0)
[6] (pending)    :unexpanded (move f0 f1)
[7] (pending)    :unexpanded (debark p0 f1)
[8] (closed)     :unexpanded (move_primitive f0 f0)
[9] (pending)    :action (move_primitive f0 f0)

Not explored (1):  4


In [41]:
debugger.command("print state")
# debugger.describe(solve_elevator)

debug:> print state
(type_member_floor f1)
(type_member_floor f0)
(type_member_floor f6)
(type_member_floor f5)
(type_member_floor f4)
(type_member_floor f3)
(type_member_floor f2)
(type_member_floor f7)
(type_member_floor f8)
(type_member_floor f9)
(type_member_object__compiled p4)
(type_member_object__compiled f1)
(type_member_object__compiled f2)
(type_member_object__compiled p3)
(type_member_object__compiled p2)
(type_member_object__compiled p1)
(type_member_object__compiled p0)
(type_member_object__compiled f9)
(type_member_object__compiled f8)
(type_member_object__compiled f7)
(type_member_object__compiled f6)
(type_member_object__compiled f5)
(type_member_object__compiled f4)
(type_member_object__compiled f3)
(type_member_object__compiled f0)
(type_member_person p0)
(type_member_person p1)
(type_member_person p2)
(type_member_person p3)
(type_member_person p4)
(goal p0)
(goal p1)
(goal p2)
(goal p3)
(goal p4)
(lift_at f0)
(origin p0 f0)
(origin p1 f0)
(origin p4 f2)
(origin p3 f

In [14]:
debugger.command("next")
tareas = debugger.next()

debug:> next
(*** 1 ***) Expanding: [0] (solve_elevator)

(*** 1 ***) Selecting a candidate task.
(*** 1 ***) Found: 1 candidates (left).
      [0] :task (solve_elevator)


In [8]:
debugger.command("help")

Something went wrong
debug:> help

(*** 1 ***) Initializating planning process...
(*** 1 ***) Initializating python...
(*** 1 ***) Depth: 1
(*** 1 ***) Selecting task to expand from agenda.
Type `help <command>' to obtain detailed information about a command.
Avaliable commands:
print          display        quit           plot
continue       undisplay      next           set
describe       break          disable        enable
nexp           eval           watch          apply
<enter> executes the last command.
